# Final Project: Functional Programming

**Laura Belizón Merchán** - 100452273  
**Jorge Lázaro Ruiz** - 100452172  
*Degree in Applied Mathematics and Computation*

## Brief summary

*Pending...*

In [1]:
-- Necessary imports
import Data.Maybe (fromJust)
import Data.List (intercalate, find)

## Types

> 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data, newtype) for each of them.

An agenda is described as a list of events, so it makes sense to center the type definitions around the characteristics of an event.

For this, we can create the different types of descriptors defined and then use them for the constructor of an event.

In [2]:
type Name = String
data Type = Personal | Health | Work | Management
data Date = Date {day :: Int, month :: String, year :: Int, workday :: Bool}
data Time = Time {start :: (Int, Int), end :: (Int, Int), duration :: Maybe Int}
newtype Participants = Participants [Name]
data Repetitions = Punctual | Daily Int | Weekday Int | Weekly Int

* `Name` is just an alias for a `String`, as there are no spectial requirements for it.
* `Type` and `Repetitions` need to be instances of class `Show` if we want to be able to print them, and since they can only take a set of values we must define them as a `data`.
* `Date` and `Time` require a constructor with multiple parameters, hence we use `data` to define them. They also have to be instances of class `Show` so we can pretty-print them.
* `Participants` could be defined as an alias for `[Name]` but in order to avoid confusion with just any other list of strings, we decided to make it into a `newtype` to distinguish it and prevent unintended operations with other lists of strings.

Because we will need to display this to the user, we declare each type as a `Show` instance and define `show` for each one.

In [3]:
instance Show Type where
    show Personal = "Personal"
    show Health = "Health"
    show Work = "Work"
    show Management = "Management"

instance Show Date where
    show (Date day month year workday) = show day ++ " " ++ month ++ " " ++ show year ++ " " ++
        if workday then "(Workday)" else ""

leadingZero :: Int -> String
leadingZero n = if n < 10 then "0" ++ show n else show n

instance Show Time where
    show (Time (startHour, startMinute) (endHour, endMinute) duration) = 
        show startHour ++ ":" ++ leadingZero startMinute ++ " - " ++ 
        show endHour ++ ":" ++ leadingZero endMinute ++ 
        maybe "" (\d -> "  (Duration: " ++ show d ++ " minutes)") duration

instance Show Participants where
    show (Participants names) = intercalate ", " names

instance Show Repetitions where
    show Punctual = "Punctual"
    show (Daily n) = "Daily " ++ show n
    show (Weekday n) = "Weekday " ++ show n
    show (Weekly n) = "Weekly " ++ show n

Because we will need to compare dates and times, we declare each one as an `Eq` instance and define `==` for each one.

In [4]:
instance Eq Date where
    (Date day1 month1 year1 _) == (Date day2 month2 year2 _) = 
        day1 == day2 && month1 == month2 && year1 == year2

instance Eq Time where
    (Time (startHour1, startMinute1) (endHour1, endMinute1) _) == 
        (Time (startHour2, startMinute2) (endHour2, endMinute2) _) = 
            startHour1 == startHour2 && startMinute1 == startMinute2 && 
            endHour1 == endHour2 && endMinute1 == endMinute2

In [5]:
instance Enum Date where
    succ dt@(Date d m y w)
        | not (rightMonth dt) || not (rightDay dt) = Date 0 "Error" 0 w
        | not (rightDay (Date (d+1) m y w)) && not (rightMonth (Date d (intToMonth (monthToInt m + 1)) y w)) = Date 1 "January" (y+1) w
        | not (rightDay (Date (d+1) m y w)) = Date 1 (intToMonth (monthToInt m + 1)) y w
        | otherwise = Date (d+1) m y w
    pred dt@(Date d m y w)
        | not (rightMonth dt) || not (rightDay dt) = Date 0 "Error" 0 w
        | not (rightDay (Date (d-1) m y w)) && not (rightMonth (Date d (intToMonth (monthToInt m - 1)) y w)) = Date 31 "December" (y-1) w
        | not (rightDay (Date (d-1) m y w)) = if m == "March" && isleap dt then Date 29 "February" y w else if m == "March" then Date 28 "February" y w else if m `elem` m31 then Date 31 (intToMonth (monthToInt m - 1)) y w else Date 30 (intToMonth (monthToInt m - 1)) y w
        | otherwise = Date (d-1) m y w
        where m31 = ["February", "April", "June", "August", "September", "November"]

rightMonth :: Date -> Bool
rightMonth (Date d m y w) = m `elem` months
    where months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

rightDay :: Date -> Bool
rightDay dt@(Date d m y w)
    | d <= 0 = False
    | m == "February" && d <= 28 = True
    | m == "February" && not (isleap dt) && d >= 29 = False
    | elem m m31 && d <= 31 = True
    | d <= 30 = True
    | otherwise = False
    where m31 = ["January", "March", "May", "July", "August", "October", "December"]

isleap :: Date -> Bool
isleap (Date d m y w)
    | mod y 400 == 0 = True
    | mod y 100 == 0 = False
    | mod y 4 == 0 = True
    | otherwise = False

monthToInt :: String -> Int
monthToInt month = fromJust $ lookup month (zip months [0..13])
    where months = ["Error", "January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December", "Error"]

intToMonth :: Int -> String
intToMonth month = fromJust $ lookup month (zip [0..13] months)
    where months = ["Error", "January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December", "Error"]

With all the types defined, all we need to do is create the type `Descriptor`, with a constructor for each field.  
After that, we can define `Event` as a list of `Descriptor`s.

In [6]:
data Descriptor =
    NameDescriptor Name
    | TypeDescriptor Type
    | DateDescriptor Date
    | TimeDescriptor Time
    | ParticipantsDescriptor Participants
    | RepetitionsDescriptor Repetitions
    deriving Show

type Event = [Descriptor]

## Event display

> 2. A function to show the events in a pretty way, following the order of descriptors as presented above.

Because the `Event` type is just a list of `Descriptor`s with no particular order, we need to sort it before printing it.

For this, we created a set of "getter" functions that find instances of a given descriptor in a list of descriptors.

In [7]:
getName :: Event -> Name
getName event = case find isNameDescriptor event of
        Just (NameDescriptor name) -> name
        _ -> error "Event has no name"
    where
        isNameDescriptor (NameDescriptor _) = True
        isNameDescriptor _ = False

getType :: Event -> Type
getType event = case find isTypeDescriptor event of
        Just (TypeDescriptor type') -> type'
        _ -> error "Event has no type"
    where
        isTypeDescriptor (TypeDescriptor _) = True
        isTypeDescriptor _ = False

getDate :: Event -> Maybe Date
getDate event = case find isDateDescriptor event of
        Just (DateDescriptor date) -> Just date
        _ -> Nothing
    where
        isDateDescriptor (DateDescriptor _) = True
        isDateDescriptor _ = False

getTime :: Event -> Maybe Time
getTime event = case find isTimeDescriptor event of
        Just (TimeDescriptor time) -> Just time
        _ -> Nothing
    where
        isTimeDescriptor (TimeDescriptor _) = True
        isTimeDescriptor _ = False

getParticipants :: Event -> Participants
getParticipants event = case find isParticipantsDescriptor event of
        Just (ParticipantsDescriptor participants) -> participants
        _ -> error "Event has no participants"
    where
        isParticipantsDescriptor (ParticipantsDescriptor _) = True
        isParticipantsDescriptor _ = False

getRepetitions :: Event -> Repetitions
getRepetitions event = case find isRepetitionsDescriptor event of
        Just (RepetitionsDescriptor repetitions) -> repetitions
        _ -> error "Event has no repetitions descriptor"
    where
        isRepetitionsDescriptor (RepetitionsDescriptor _) = True
        isRepetitionsDescriptor _ = False

In [8]:
printEvent :: Event -> IO ()
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ show (getName event)
    putStrLn $ "  Type: " ++ show (getType event)
    putStrLn $ "  Date: " ++ maybe "Unknown" show (getDate event)
    putStrLn $ "  Time: " ++ maybe "Unknown" show (getTime event)
    putStrLn $ "  Participants: " ++ show (getParticipants event)
    putStrLn $ "  Repetitions: " ++ show (getRepetitions event)

-- Example usage
eventName = NameDescriptor "Example"
eventType = TypeDescriptor Work
eventDate = DateDescriptor (Date 6 "December" 2023 True)
eventTime = TimeDescriptor (Time (10, 30) (12, 0) (Just 90))
eventParticipants = ParticipantsDescriptor (Participants ["John", "Alice", "Bob"])
eventRepetitions = RepetitionsDescriptor (Weekly 2)
exampleEvent = [eventName, eventType, eventDate, eventTime, eventParticipants, eventRepetitions]
printEvent exampleEvent

Event:
  Name: "Example"
  Type: Work
  Date: 6 December 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

## Utility functions

> 3. An `isWorkingDate Date [Date]` function that receives a `Date` and a list of holidays and returns if the day is weekday or weekend/holiday by considering leap years and the fact that 1<sup>st</sup> January 2022 was Saturday.

For this function, we decided to implement the Doomsday algorithm, which allows to compute the day of the week of any given date.

For this century, the anchor day must be a tuesday, so we chose the 4th of January of 2022. If we wanted to compute a date on a different century, we would need to change the anchorday (but we assume for now we wont schedule events on the next or the past centuries).

In [42]:
calculateDoomsday :: Date -> Int
calculateDoomsday dt@(Date d m y w) = mod s 7
    where
        s = step1 + step2 + step3 + anchorday
        step1 = div (mod y 2000) 12    -- times the number 12 fit as a whole into the two last digits of the year number
        step2 = mod y 2000 - (step1 * 12)    -- difference between the two last digits of the year number and the product of the multiples of 12 from step1
        step3 = div step2 4    -- times the number 4 fit into the result of step2
        anchorday = 2    -- Use as anchor day the 4th of January of 2022 (tuesday)

Once we have found out which day of the week is the Doomsday of the year asked, we need to generate a list with all this Doomsdays. That is, the days of the given year which fall for certain in the calculated day of the week.

In [10]:
generateDoomsdays :: Date -> [Date]
generateDoomsdays dt@(Date d m y w)
    | isleap dt = [Date 4 "January" y w, Date 29 "February" y w, Date 7 "March" y w, Date 4 "April" y w, Date 9 "May" y w, Date 6 "June" y w, Date 11 "July" y w, Date 8 "August" y w, Date 5 "September" y w, Date 10 "October" y w, Date 7 "November" y w, Date 12 "December" y w]
    | otherwise = [Date 3 "January" y w, Date 28 "February" y w, Date 7 "March" y w, Date 4 "April" y w, Date 9 "May" y w, Date 6 "June" y w, Date 11 "July" y w, Date 8 "August" y w, Date 5 "September" y w, Date 10 "October" y w, Date 7 "November" y w, Date 12 "December" y w]

The next step is to implement a function which calculates how many days fall between the date we are seeking and the closest Doomsday. In order to do this, we implemented the function recursively.

In [29]:
doomsday :: Date -> Int
doomsday dt
    | dt `elem` dd = calculateDoomsday dt
    | otherwise = itdoomsday 1 (succ dt)
    where dd = generateDoomsdays dt

itdoomsday :: Int -> Date -> Int
itdoomsday i dt
    | dt `elem` dd = calculateDoomsday dt - i
    | otherwise = itdoomsday (i+1) (succ dt)
    where dd = generateDoomsdays dt

Finally, we can check if the date is a holiday, Saturday (6) or Sunday (0), and if it is, return `False`.

In [36]:
isWorkingDate :: Date -> [Date] -> Bool
isWorkingDate date holidays
    | date `elem` holidays = False
    | mod (doomsday date) 7 == 6 = False
    | mod (doomsday date) 7 == 0 = False
    | otherwise = True

In [51]:
isWorkingDate (Date 12 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 10 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 25 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 9 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 23 "February" 2024 True) [Date 25 "December" 2023 True]

True

False

False

False

True